In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [20]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data,
                    columns=boston_dataset.feature_names)
features = data.drop(["INDUS", "AGE"], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=["PRICE"])

In [13]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

property_stats = features.mean().values.reshape(1, 11)

In [25]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [26]:
def get_log_esitmate(nr_rooms, students_per_classroom,
                    next_to_river=False, high_confidence=True):
    
    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calculating Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95 # Wide range interval
    else: 
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68 # Narrow range interval 
    
    return log_estimate, upper_bound, lower_bound, interval

In [29]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_esitmate(9, students_per_classroom=15,
                                              next_to_river=False, high_confidence=False)

# Convert to todays dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_high = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round the dollar values to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_high = np.around(dollar_high, -3)
rounded_low = np.around(dollar_low, -3)

print(f"The estimated property value is {rounded_est}.")
print(f"At {conf}% confidence the valuation range is")
print(f"USD {rounded_low} at the lower end to USD {rounded_high} at the high end.")

The estimated property value is 827000.0.
At 68% confidence the valuation range is
USD 685000.0 at the lower end to USD 997000.0 at the high end.


In [34]:
def get_dollars_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classrom for he school in the area.
    chas -- True if the property is next to the river, False otherwise.
    large_range -- True for a 95% prediction interval, False for a 68% interval.
    
    """
    
    if rm < 1 or ptratio < 1 :
        print("That is unrealistic. Try again.")
        return
    
    log_est, upper, lower, conf = get_log_esitmate(rm, ptratio,
                                                  next_to_river=chas,
                                                  high_confidence=large_range)

    # Convert to todays dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_high = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_high = np.around(dollar_high, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f"The estimated property value is {rounded_est}.")
    print(f"At {conf}% confidence the valuation range is")
    print(f"USD {rounded_low} at the lower end to USD {rounded_high} at the high end.")

In [32]:
get_dollars_estimate(rm=2, ptratio=30, chas=True)

The estimated property value is 278000.0.
At 95% confidence the valuation range is
USD 191000.0 at the lower end to USD 404000.0 at the high end.


In [35]:
import boston_valuation as val
val.get_dollars_estimate(5, 12, True)

The estimated property value is 715000.0.
At 95% confidence the valuation range is
USD 491000.0 at the lower end to USD 1040000.0 at the high end.
